# Notebook responsável por apresentar os comandos básicos para manipulação de dados no mongo

## Crud com apenas 1 documento
- Inserir 1 documento **simples**
- Recuperar 1 documento **simples**
- Atualizar 1 documento **simples**
- Excluir 1 documento **simples**

## Bibliotecas importantes

In [1]:
from pymongo import MongoClient
import pprint
import numpy as np
import pandas as pd

# para geração de dados fake
from faker import Faker


## Funções uteis

In [2]:
def criar_pessoa():
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    
    idade = np.random.randint(10, 30)
    
    pessoa = {
        'cpf': fake.cpf(),
        'nome': f'{primeiro_nome} {ultimo_nome}' ,
        'idade': idade ,
    }        
    pessoa['email']= f'{primeiro_nome.lower()}.{ultimo_nome.lower()}@{fake.free_email_domain()}'
    
    return pessoa

## Configurações

In [3]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de 1 documento

### Conectando ao MongoDB

In [4]:
client = MongoClient('localhost', 27017)
# Se a base de dados não existir, ela é criada automagicamente :-)
db = client['techshot_mongo01']

### Exemplo - Criação de um documento simples

In [5]:
pessoa = criar_pessoa()
pp.pprint(pessoa)

{'cpf': '630.429.875-74',
 'email': 'maria clara.teixeira@hotmail.com',
 'idade': 28,
 'nome': 'Maria Clara Teixeira'}


In [6]:
pd.DataFrame.from_records([pessoa])

,cpf,nome,idade,email
0,630.429.875-74,Maria Clara Teixeira,28,maria clara.teixeira@hotmail.com


### Inserindo o documento no banco de dados

In [7]:
# se a coleção não existir, ela é criada automagicamente :-)
colecao_pessoa = db['pessoas']
pessoa_id = colecao_pessoa.insert_one(pessoa).inserted_id
pessoa_id

ObjectId('625fee0298615463b5756e43')

## Recuperando o documento inserido

#### Recuperar pelo ID

In [8]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)

{'_id': ObjectId('625fee0298615463b5756e43'),
 'cpf': '630.429.875-74',
 'email': 'maria clara.teixeira@hotmail.com',
 'idade': 28,
 'nome': 'Maria Clara Teixeira'}


#### Recuperando por algum outro atributo

In [10]:
criterio = {
    'idade': {'$eq':pessoa['idade']}
}
pp.pprint(criterio)


{'idade': {'$eq': 28}}


In [11]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

{'_id': ObjectId('625fee0298615463b5756e43'),
 'cpf': '630.429.875-74',
 'email': 'maria clara.teixeira@hotmail.com',
 'idade': 28,
 'nome': 'Maria Clara Teixeira'}


#### Recuperando por algum outro atributo

In [12]:
criterio = {
    'nome': {'$eq':pessoa['nome']}
}
pp.pprint(criterio)


{'nome': {'$eq': 'Maria Clara Teixeira'}}


In [13]:
p = colecao_pessoa.find_one(criterio)
pp.pprint(p)

{'_id': ObjectId('625fee0298615463b5756e43'),
 'cpf': '630.429.875-74',
 'email': 'maria clara.teixeira@hotmail.com',
 'idade': 28,
 'nome': 'Maria Clara Teixeira'}


In [14]:
criterio = {
    'nome': {'$eq':'nome_estranho_que_não_será_criado'}
}
pp.pprint(criterio)

{'nome': {'$eq': 'nome_estranho_que_não_será_criado'}}


In [15]:
p = colecao_pessoa.find_one(criterio)
p, type(p)

(None, NoneType)

## Atualizar o documento

In [16]:
criterio = {
    "_id":pessoa_id}

novo_valor = {
    "$set": { "idade": 50 }
}
pp.pprint(novo_valor)


{'$set': {'idade': 50}}


In [17]:
r = colecao_pessoa.update_one(criterio, novo_valor)
r

In [18]:
# ref.: https://pymongo.readthedocs.io/en/stable/api/pymongo/results.html
r.matched_count

1

In [19]:
r.modified_count

1

In [21]:
r.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [22]:
r.upserted_id

In [24]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)

{'_id': ObjectId('625fee0298615463b5756e43'),
 'cpf': '630.429.875-74',
 'email': 'maria clara.teixeira@hotmail.com',
 'idade': 50,
 'nome': 'Maria Clara Teixeira'}


## Atualizar o documento (que não existe)

In [25]:
criterio = {
    "_id":'123'}

novo_valor = {
    "$set": { "idade": 50 }
}
pp.pprint(novo_valor)

{'$set': {'idade': 50}}


In [26]:
r = colecao_pessoa.update_one(criterio, novo_valor)
r

In [27]:
r.matched_count

0

In [28]:
r.modified_count

0

In [29]:
r.raw_result

{'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}

In [30]:
r = colecao_pessoa.update_one(criterio, novo_valor, upsert=True)
r

In [31]:
r.raw_result

{'n': 1,
 'upserted': '123',
 'nModified': 0,
 'ok': 1.0,
 'updatedExisting': False}

In [32]:
r.upserted_id

'123'

In [34]:
p = colecao_pessoa.find_one({"_id":r.upserted_id})
pp.pprint(p)

{'_id': '123', 'idade': 50}


## Remover o documento

In [35]:
criterio = {
    "_id":pessoa_id}
pp.pprint(criterio)

{'_id': ObjectId('625fee0298615463b5756e43')}


In [36]:
r = colecao_pessoa.delete_one(criterio)

In [37]:
r.raw_result

{'n': 1, 'ok': 1.0}

In [38]:
p = colecao_pessoa.find_one({"_id":pessoa_id})
pp.pprint(p)

None
